# ufunc 对象

**Numpy** 有两种基本对象：`ndarray (N-dimensional array object)` 和 `ufunc (universal function object)`。`ndarray` 是存储单一数据类型的多维数组，而 `ufunc` 则是能够对数组进行处理的函数。

例如，我们之前所接触到的二元操作符对应的 **Numpy** 函数，如 `add`，就是一种 `ufunc` 对象，它可以作用于数组的每个元素。

In [2]:
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
a = np.array([0,1,2])
b = np.array([2,3,4])

np.add(a, b)

array([2, 4, 6])

查看支持的方法：

In [3]:
dir(np.add) # 因为np.add 也是个对象，所以就会有相关的对象方法

['__call__',
 '__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__name__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'accumulate',
 'at',
 'identity',
 'nargs',
 'nin',
 'nout',
 'ntypes',
 'outer',
 'reduce',
 'reduceat',
 'signature',
 'types']

除此之外，大部分能够作用于数组的数学函数如三角函数等，都是 `ufunc` 对象。

特别地，对于二元操作符所对应的 `ufunc` 对象，支持以下方法：

## reduce 方法

    op.reduce(a)

将`op`沿着某个轴应用，使得数组 `a` 的维数降低一维。

add 作用到一维数组上相当于求和：

$$
\begin{align}
y & = add.recuce(a) \\
& = a[0] + a[1] + ... + a[N-1] \\
& = \sum_{n=0}^{N-1} a[n]
\end{align}
$$

In [3]:
a = np.array([1,2,3,4])

np.add.reduce(a)
# cc：这个挺关键。判断某个ufunc有没有reduce可以用 dir(add)方法来查看。

10

In [4]:
# cc
import numpy as np
a2 = np.array([[1,2,3,4],
             [5,6,7,8]])
a2
np.add.reduce(a2)

# 不用reduce会出现报错。所以这既是reduce的作用
np.add(a2)

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

array([ 6,  8, 10, 12])

ValueError: invalid number of arguments

In [6]:
np.add.reduce(a2, axis=1)

array([10, 26])

In [6]:
np.sum(a) # 所以和上面是等价的， 但是不可以np.add(a)，因为add 是方法 不是函数


10

In [7]:
np.sum(a2, axis=1)

array([10, 26])

多维数组默认只按照第一维进行运算：

In [7]:
a = np.array([[1,2,3],[4,5,6]])

np.add.reduce(a)

array([5, 7, 9])

In [8]:
np.sum(a, axis=0) 

np.sum(a) 


array([5, 7, 9])

21

指定维度：

In [11]:
np.add.reduce(a, 1)

array([ 6, 15])

In [13]:
np.sum(a,axis=1)

array([ 6, 15])

作用于字符串：

In [7]:
a = np.array(['ab', 'cd', 'ef'], np.object)

np.add.reduce(a)
# cc：之前我们看到reduce是用来降维的，如果已经是一维，那这个时候连这1维都被降维了，直接成了“字符串”；
# 然后就有“字符串”相加的操作。

'abcdef'

逻辑运算：

In [8]:
a = np.array([1,1,0,1])

np.logical_and.reduce(a)

False

In [9]:
np.logical_or.reduce(a)

True

## accumulate 方法

    op.accumulate(a)

`accumulate` 可以看成保存 `reduce` 每一步的结果所形成的数组。

$$
\begin{align}
y & = add.accumulate(a) \\
& = \left[\sum_{n=0}^{0} a[n], \sum_{n=0}^{1} a[n], ..., \sum_{n=0}^{N-1} a[n]\right]
\end{align}
$$

与之前类似：

In [8]:
# cc：回顾之前reduce操作
a = np.array([1,2,3,4])
np.add.reduce(a)

10

In [10]:
a = np.array([1,2,3,4])

np.add.accumulate(a)
# cc：求累计量还是不错的。

array([ 1,  3,  6, 10])

In [11]:
a = np.array(['ab', 'cd', 'ef'], np.object)

np.add.accumulate(a)

array(['ab', 'abcd', 'abcdef'], dtype=object)

In [12]:
a = np.array([1,1,0,1])

np.logical_and.accumulate(a)

array([ True,  True, False, False], dtype=bool)

In [13]:
np.logical_or.accumulate(a)

array([ True,  True,  True,  True], dtype=bool)

## reduceat 方法

    op.reduceat(a, indices)

`reduceat` 方法将操作符运用到指定的下标上，返回一个与 `indices` 大小相同的数组：

$$
\begin{align}
y & = add.reduceat(a, indices) \\
& = \left[\sum_{n=indice[0]}^{indice[1]-1} a[n], \sum_{n=indice[1]}^{indice[2]-1} a[n], ..., \sum_{n=indice[-1]}^{N-1} a[n]\right]
\end{align}
$$

In [14]:
a = np.array([0, 10, 20, 30, 40, 50])
indices = np.array([1,4])

np.add.reduceat(a, indices)

array([60, 90])

这里，`indices` 为 `[1, 4]`，所以 `60` 表示从下标1（包括）加到下标4（不包括）的结果，`90` 表示从下标4（包括）加到结尾的结果。

## outer 方法

    op.outer(a, b)

对于 `a` 中每个元素，将 `op` 运用到它和 `b` 的每一个元素上所得到的结果：

In [15]:
a = np.array([0,1])
b = np.array([1,2,3])

np.add.outer(a, b)

array([[1, 2, 3],
       [2, 3, 4]])

注意有顺序的区别：

In [16]:
np.add.outer(b, a)

array([[1, 2],
       [2, 3],
       [3, 4]])